In [1]:
################################################################################
# Preparar entorno de trabajo
################################################################################
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=22b06660414e49a31670cf5f44dde6e18edb6fc21a8daba81958e90433bc1ddc
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
################################################################################
# Preparar entorno de trabajo
################################################################################
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# IGNORAR WARNINGS
import warnings
warnings.filterwarnings('ignore')

# Importar datos desde Drive
from google.colab import drive
drive.mount('/content/drive')

# Pandas
import pandas as pd

Mounted at /content/drive


In [3]:
################################################################################
# CREAMOS EL SPARK SESSION
################################################################################
spark = SparkSession.builder.appName("Nutricion").getOrCreate()

In [4]:
################################################################################
# Datos Open Food Facts
# URL = https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv
################################################################################
df_datos = spark.read.options(header="True", inferSchema='True', delimiter='\t').csv("/content/drive/MyDrive/Datos TFM/en.openfoodfacts.org.products.csv")

In [5]:
################################################################################
# DATAFRAME A SQL
################################################################################
df_datos.createOrReplaceTempView("openfoodfacts")

In [6]:
################################################################################
# Cantidad de productos
################################################################################
df_datos.select(df_datos.product_name).count()

2850798

In [7]:
################################################################################
# Cantidad de productos distintos
################################################################################
df_datos.select(df_datos.product_name).distinct().count()

1714180

In [8]:
################################################################################
# Schema de datos
################################################################################
df_datos.printSchema()

root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- categories_tags: string (nullable = true)
 |-- categories_en: string (nullable = true)
 |-- origins: string (nullable = true)
 |-- origins_tags: strin

In [9]:
################################################################################
# Describir cada una de las columnas y determinar si son cualitativas o cuantitativas
################################################################################

# Obtén el esquema del DataFrame
schema = df_datos.schema

# Crea una lista vacía para almacenar los nombres de las columnas
column_names = []

# Crea dos listas vacías para almacenar los tipos de datos y los tipos de variable de cada columna
data_types = []
variable_types = []

# Itera sobre cada columna del esquema
for field in schema.fields:
    # Obtén el nombre de la columna
    column_name = field.name
    # Agrega el nombre de la columna a la lista correspondiente
    column_names.append(column_name)
    
    # Obtén el tipo de datos de la columna
    data_type = str(field.dataType)

    # Agrega el tipo de datos a la lista correspondiente
    data_types.append(data_type)
    
    # Determina si la columna es cualitativa o cuantitativa
    if data_type.startswith("StringType"):
        variable_type = "Cualitativa"
    else:
        variable_type = "Cuantitativa"
    # Agrega el tipo de variable a la lista correspondiente
    variable_types.append(variable_type)

# Crea un DataFrame para mostrar los resultados
result_df = spark.createDataFrame(zip(column_names, data_types, variable_types), ["Columna", "Tipo de datos", "Tipo de variable"])
result_df.show(n=300, truncate=False, vertical=False)

+-----------------------------------------------------+---------------+----------------+
|Columna                                              |Tipo de datos  |Tipo de variable|
+-----------------------------------------------------+---------------+----------------+
|code                                                 |DoubleType()   |Cuantitativa    |
|url                                                  |StringType()   |Cualitativa     |
|creator                                              |StringType()   |Cualitativa     |
|created_t                                            |IntegerType()  |Cuantitativa    |
|created_datetime                                     |TimestampType()|Cuantitativa    |
|last_modified_t                                      |IntegerType()  |Cuantitativa    |
|last_modified_datetime                               |TimestampType()|Cuantitativa    |
|last_modified_by                                     |StringType()   |Cualitativa     |
|product_name        

In [10]:
################################################################################
# Obtener los productos de tipo:
# - Seitan
# - Soja texturizada
# - Tofu
################################################################################
df_products_distinct = spark.sql('''SELECT DISTINCT * FROM openfoodfacts 
                                                    WHERE 
                                                        product_name LIKE "%seitan%" OR 
                                                        (product_name LIKE "%soja%" AND product_name LIKE "%tex%") OR
                                                        product_name LIKE "%tofu%"  ''')
df_products_distinct.show(truncate=False)

+---------------+-------------------------------------------------------------------------------------------------------------------------+-----------------+----------+-------------------+---------------+----------------------+-----------------+---------------------------------------------------------+------------------------+------------+--------+--------------------+-----------------------+--------------------+--------------+------------------------+----------------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+-------+------------+----------+--------------------+-------------------------+-----------------------------------------+--------------------------------

In [11]:
################################################################################
# Convertir el DataFrame de PySpark a un DataFrame de pandas
################################################################################
pandas_df = df_products_distinct.toPandas()

In [12]:
################################################################################
# Funciones de limpieza de datos (DataFrame Pandas)
################################################################################
def datos_a_mayusculas(df):
    
    df = df.apply(lambda x: x.str.upper() if x.dtype == "object" else x)
    
    return df
################################################################################
################################################################################
################################################################################
def eliminar_nulos(df):
    df = df.dropna(axis=0, how='all')
    
    return df
################################################################################
################################################################################
################################################################################
def eliminar_duplicados(df):
    df = df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
    
    return df
################################################################################
################################################################################
################################################################################
def reemplazar_caracteres(df):
    df = df.str.upper()
    df = df.str.replace('/',' ')
    df = df.str.replace(',',' ')
    df = df.str.replace('-',' ')
    df = df.str.replace(' ','_')
    df = df.str.replace('Á','A')
    df = df.str.replace('É','E')
    df = df.str.replace('Í','I')
    df = df.str.replace('Ó','O')
    df = df.str.replace('Ú','U')
    df = df.str.replace('Ü','U')
    df = df.str.replace('Ñ','N')

    return df

In [13]:
################################################################################
#En este paso se convienten los datos contenidos en los dataframe a mayúsculas, 
# esto con el fin de estandarizarlos.
################################################################################

pandas_df = datos_a_mayusculas(pandas_df)

In [14]:
################################################################################
# Se eliminan las filas en las que todos los valores son nulos, 
# ya que se trata de filas que no contribuyen en los análisis que se harán en las etapas siguientes.
################################################################################

pandas_df = eliminar_nulos(pandas_df)

In [15]:
################################################################################
# identificar las columnas que son de tipo string y las columnas que son numéricas.
################################################################################

# Identificar columnas que son de tipo string
str_cols = pandas_df.select_dtypes(include=['object']).columns

# Identificar columnas que son numéricas
num_cols = pandas_df.select_dtypes(include=['float64', 'int64']).columns

In [16]:
################################################################################
#Se reemplazan los casos en que existe un NaN en cada uno de los dataframe creados por un dato: 
# - "vacío" cuando es String.
# - "0" cuando es Numérica.
################################################################################
# Reemplazar NaN en columnas de tipo string con ''
pandas_df[str_cols] = pandas_df[str_cols].fillna('')

# Reemplazar NaN en columnas numéricas con 0
pandas_df[num_cols] = pandas_df[num_cols].fillna(0)

In [17]:
################################################################################
#Se eliminan en este paso los datos duplicados, si los hubiera, en el dataframe.
################################################################################

pandas_df = eliminar_duplicados(pandas_df)

In [18]:
################################################################################
# Con el fin de no tener problemas al momento de invocar las columnas, 
# se estandarizan los nombres que actualmente existen en el dataframe.
################################################################################

pandas_df.columns = reemplazar_caracteres(pandas_df.columns)

In [19]:
################################################################################
# DataFrame Pre-Procesado
################################################################################
pandas_df

,CODE,URL,CREATOR,CREATED_T,CREATED_DATETIME,LAST_MODIFIED_T,LAST_MODIFIED_DATETIME,LAST_MODIFIED_BY,PRODUCT_NAME,ABBREVIATED_PRODUCT_NAME,...,CARBON_FOOTPRINT_FROM_MEAT_OR_FISH_100G,NUTRITION_SCORE_FR_100G,NUTRITION_SCORE_UK_100G,GLYCEMIC_INDEX_100G,WATER_HARDNESS_100G,CHOLINE_100G,PHYLLOQUINONE_100G,BETA_GLUCAN_100G,INOSITOL_100G,CARNITINE_100G
0,1.614137e+10,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/0016...,KILIWEB,1608648987,2020-12-22 14:56:27,1672083336,2022-12-26 19:35:36,ROBOTO-APP,TRADITIONAL SEITAN,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.111011e+10,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/0011...,KILIWEB,1665097434,2022-10-06 23:03:54,1675091216,2023-01-30 15:06:56,WOLFGANG8741,BACKED TOFU,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.674131e+10,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/0016...,KILIWEB,1608667651,2020-12-22 20:07:31,1664808685,2022-10-03 14:51:25,ROBOTO-APP,TRADITIONAL SEITAN,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.622993e+10,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/0016...,KILIWEB,1620644544,2021-05-10 11:02:24,1682692507,2023-04-28 14:35:07,ROBOTO-APP,FROZEN FISH TOFU,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.111020e+10,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/0011...,ORG-DATABASE-USDA,1587664560,2020-04-23 17:56:00,1587664561,2020-04-23 17:56:01,ORG-DATABASE-USDA,EDAMAME + TOFU PONZU BOWL WITH CABBAGE AND PIC...,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,9.788480e+15,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/9788...,ELCOCO,1576245122,2019-12-13 13:52:02,1579287843,2020-01-17 19:04:03,MUSARANA,"BIO BURGER VEGETAL SEITAN, ALGAS Y BERENJENAS",,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1202,9.415837e+12,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/9415...,KILIWEB,1619830127,2021-05-01 00:48:47,1682772446,2023-04-29 12:47:26,ROBOTO-APP,PEKING MARINATED TOFU,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1203,9.788480e+15,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/9788...,ELCOCO,1576588642,2019-12-17 13:17:22,1579285917,2020-01-17 18:31:57,MUSARANA,"BIO BURGER VEGETAL SEITAN, ALGAS Y BERENJENAS",,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1204,9.421004e+12,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/9421...,KILIWEB,1598832500,2020-08-31 00:08:20,1679662003,2023-03-24 12:46:43,ITSJUSTRUBY,ORGANIC TOFU,,...,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
################################################################################
# Contador de datos por columna
################################################################################

pandas_df_count = pandas_df.count()
pandas_df_count

CODE                  1206
URL                   1206
CREATOR               1206
CREATED_T             1206
CREATED_DATETIME      1206
                      ... 
CHOLINE_100G          1206
PHYLLOQUINONE_100G    1206
BETA_GLUCAN_100G      1206
INOSITOL_100G         1206
CARNITINE_100G        1206
Length: 201, dtype: int64

In [21]:
################################################################################
# Descripción de datos por columna
################################################################################

pandas_df.describe(include='all')

,CODE,URL,CREATOR,CREATED_T,CREATED_DATETIME,LAST_MODIFIED_T,LAST_MODIFIED_DATETIME,LAST_MODIFIED_BY,PRODUCT_NAME,ABBREVIATED_PRODUCT_NAME,...,CARBON_FOOTPRINT_FROM_MEAT_OR_FISH_100G,NUTRITION_SCORE_FR_100G,NUTRITION_SCORE_UK_100G,GLYCEMIC_INDEX_100G,WATER_HARDNESS_100G,CHOLINE_100G,PHYLLOQUINONE_100G,BETA_GLUCAN_100G,INOSITOL_100G,CARNITINE_100G
count,1.206000e+03,1206,1206,1.206000e+03,1206,1.206000e+03,1206,1206,1206,1206,...,1206.0,1206.000000,1206.0,1206.0,1206.0,1206.0,1.206000e+03,1206.0,1206.0,1206.0
unique,NaN,1206,106,NaN,1203,NaN,1198,98,1030,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,HTTP://WORLD-EN.OPENFOODFACTS.ORG/PRODUCT/0016...,KILIWEB,NaN,2017-03-09 12:30:12,NaN,2022-02-10 17:41:43,ROBOTO-APP,FIRM TOFU,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,749,NaN,2,NaN,2,352,15,1206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,2012-06-02 14:05:31,NaN,2017-03-09 13:00:41,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2023-04-20 12:34:35,NaN,2023-05-01 14:28:54,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,7.049968e+20,NaN,NaN,1.585155e+09,NaN,1.646692e+09,NaN,NaN,NaN,NaN,...,0.0,0.674129,0.0,0.0,0.0,0.0,1.169154e-08,0.0,0.0,0.0
std,1.555723e+22,NaN,NaN,6.773751e+07,NaN,3.682643e+07,NaN,NaN,NaN,NaN,...,0.0,3.828798,0.0,0.0,0.0,0.0,4.060182e-07,0.0,0.0,0.0
min,3.246000e+03,NaN,NaN,1.338646e+09,NaN,1.489064e+09,NaN,NaN,NaN,NaN,...,0.0,-9.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
25%,8.415924e+11,NaN,NaN,1.549523e+09,NaN,1.627435e+09,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0


In [22]:
################################################################################
# Exportar el DataFrame de pandas como un archivo CSV
################################################################################
pandas_df.to_csv("alimentos.csv", index=False)